Importing libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import linalg

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

Enter dimensions of soccer field

In [2]:
goal_width=7.32
goal_height=2.44
goal_area_1=5.5
goal_area_2=18.32
penalty_area_1=16.5
penalty_area_2=40.32
corner_arc=1
flag_height=1.5
field_1=65
field_2=105

Enter 3D point coordinates - Goal-> Goal area -> Penalty area -> Corners -> Corner radius points -> Flag extremity

In [3]:
        # Goal
u_field=np.array([[goal_width/2,0,0],[goal_width/2,goal_height,0],   
        [-goal_width/2,goal_height,0],[goal_width/2,0,0],            
        # Goal
        [goal_area_2/2,0,0],[goal_area_2/2,0,goal_area_1],           
        # Goal area
        [-goal_area_2/2,0,goal_area_1],[-goal_area_2/2,0,0],
        # Goal area
        [penalty_area_2/2,0,0],[penalty_area_2/2,0,penalty_area_1],
        # Penalty area
        [-penalty_area_2/2,0,penalty_area_1],[-penalty_area_2/2,0,0],
        # Penalty area
        [field_1/2,0,0],[-field_1/2,0,0], # Corners
        [field_1/2-corner_arc,0,0],[-field_1/2+corner_arc,0,0],
        # Corner radius x
        [field_1/2,0,corner_arc],[-field_1/2,0,corner_arc],
        # Corner radius y
        [field_1/2,flag_height,0],[-field_1/2,flag_height,0]])
        # Flag extremities

Keep points that are visible in the image buffer

In [ ]:
    # Goal
u_w=np.array([[goal_width/2,0,0],[goal_width/2,goal_height,0],
    # Goal
    [-goal_width/2,goal_height,0],[-goal_width/2,0,0],
    # Goal area
    [goal_area_2/2,0,0],[goal_area_2/2,0,goal_area_1],
    # Goal area
    [-goal_area_2/2,0,goal_area_1],[-goal_area_2/2,0,0],
    # Penalty area
    [-penalty_area_2/2,0,0],
    # Corners
    [-field_1/2,0,0],
    # Corner radius x
    [-field_1/2+corner_arc,0,0],
    # Corner radius y
    [-field_1/2,0,corner_arc],
    # Flag extremities
    [-field_1/2,flag_height,0]])          

# Number of visible points
N=np.shape(u_w)[0]
# Transformation to homogeneous coordinates
u_w= np.vstack((np.transpose(u_w),np.ones(N)))

Show 3D cloud of points

In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(u_w[:,0],u_w[:,2],u_w[:,1])
#ax.set_xlim(-35, 35)
#ax.set_zlim(-1, 60)
#ax.set_ylim(-5,65)

Get image buffer points manually

In [ ]:
img=mpimg.imread('goal.jpg')
plt.imshow(img)
# Image buffer points from mouse clicks
u_im = np.transpose(np.array(plt.ginput(N,timeout=-1))) 
u_im = np.vstack((u_im,np.ones(N))) 
plt.close()

In [ ]:
print(u_im)

Show chosen points

In [ ]:
plt.imshow(img)
plt.scatter(u_im[0,:],u_im[1,:])
plt.show()

Centering and scaling

3D points

In [ ]:
s_w    =1./(np.std(u_w,1,keepdims=True)[0:3])# Scaling
t_w    =-(np.mean(u_w,1,keepdims=True)[0:3]) # Translation
diag_s =np.diag(np.squeeze(s_w))
# Stacking scaling and translation
T_w    =np.array(np.hstack((diag_s,t_w*s_w)))
T_w    =np.vstack((T_w,np.array([0,0,0,1])))
print(T_w)

Image buffer points

In [ ]:
s_im   =1./(np.std(u_im,1,keepdims=True)[0:2]) # Scaling
t_im   =-(np.mean(u_im,1,keepdims=True)[0:2])  # Translation
diag_s =np.diag(np.squeeze(s_im))
# Stacking scaling and translation
T_im   =np.array(np.hstack((diag_s,t_im*s_im)))
T_im   =np.vstack((T_im,np.array([0,0,1])))
print(T_im)

Transformed 3D points and buffer points

In [ ]:
u_w_t = np.dot(T_w,u_w)   # Transformed 3D points
u_im_t= np.dot(T_im,u_im) # Transformed image buffer points

Construction of the A matrix for DLT

In [ ]:
u_w_i=np.transpose(u_w_t[:,0].reshape(-1,1))
A_i_1=np.hstack((np.zeros((1,4)),-u_w_i,u_im_t[1,0]*u_w_i))
A_i_2=np.hstack((u_w_i,np.zeros((1,4)),-u_im_t[0,0]*u_w_i))
A    = np.vstack((A_i_1,A_i_2))
# Stacking matrices for each point correspondence
for i in range(1,N):    
    u_w_i=np.transpose(u_w_t[:,i].reshape(-1,1))
    A_i_1=np.hstack((np.zeros((1,4)),-u_w_i,u_im_t[1,i]*u_w_i))
    A_i_2=np.hstack((u_w_i,np.zeros((1,4)),-u_im_t[0,i]*u_w_i))
    A    = np.vstack((A,A_i_1,A_i_2))

Solution using the SVD

In [ ]:
U,S,V = np.linalg.svd(A) # SVD
m_t   = V[11,:]          # Last row of V, 
                         # since python's V is the transpose
                         # of the SVD's V 

Reshape to get M_t

In [ ]:
M_t   = m_t.reshape(3,4)

Transform to get M

In [ ]:
M = np.dot(np.linalg.inv(T_im),np.dot(M_t,T_w))

Rescale to get three first elements of third row with unitary norm

In [ ]:
M = M/np.linalg.norm(M[2,0:3])
print(M)

Getting Intrinsic and rotation matrices with RQ decomposition

In [ ]:
M_int, R = linalg.rq(M[:,0:3])
print(M_int)
print(R)

Change signs of columns of M_int
and corresponding rows of R if necessary

In [ ]:
M_int[:,0] = -M_int[:,0]
R[0,:] = -R[0,:]
M_int[:,2] = -M_int[:,2]
R[2,:] = -R[2,:]
print(M_int)
print(R)

Checking determinant of the rotation matrix

In [ ]:
print(np.linalg.det(R))

Project points on the image buffer using the estimated camera matrix

In [ ]:
u_sim=np.dot(M,u_w)
u_sim=u_sim/u_sim[2,:]

plt.imshow(img)
plt.scatter(u_sim[0,:],u_sim[1,:])
plt.show()

Evaluate translation vector

In [ ]:
t=np.linalg.solve(M_int,M[:,3])
print(-np.dot(R.T,t))